In [4]:
import requests
from bs4 import BeautifulSoup
import sqlite3

%run Definitions.py
from datetime import datetime, timedelta
import re
import os

In [5]:
# conn = sqlite3.connect("/Users/ralph/Biotech/BiotechDatabase.db") 

# def create_tables():
#     # create a cursor
#     cur = conn.cursor()

#     # create tables
#     create_table_commands = (
#         """
#             CREATE TABLE filings (
#                 filing_id varchar(255) PRIMARY KEY,
#                 cik int,
#                 filer_name varchar(255),
#                 period_of_report date
#             )
#         """,
#         """
#             CREATE TABLE holdings (
#                 filing_id varchar(255),
#                 name_of_issuer varchar(255),
#                 cusip varchar(255),
#                 title_of_class varchar(255),
#                 value bigint,
#                 shares int,
#                 put_call varchar(255)
#             )
#         """, 
#         """
#             CREATE TABLE holding_infos (
#                 cusip varchar(255),
#                 security_name varchar(255),
#                 ticker var(50),
#                 exchange_code varchar(10),
#                 security_type varchar(50)
#             )
#         """)

#     # create table one by one
#     for command in create_table_commands:
#         cur.execute(command)
    
#     # close cursor
#     cur.close()
    
#     # make the changes to the database persistent
#     conn.commit()
#     conn.close()



# create_tables()

In [6]:
#V2:
import requests
import time
import os
import requests
from bs4 import BeautifulSoup

def doGet(url):
    s = requests.Session()
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'Sec-CH-UA': 'Examplary Browser',
        'Sec-CH-UA-Mobile': '?0',
        'Sec-CH-UA-Platform': "Windows",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "authority": "www.sec.gov",
        "method": "GET",
        "path": "/Archives/edgar/data/59478/000120919121046268/0001209191-21-046268-index.htm",
        "scheme": "https",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip deflate br",
        "accept-language": "en-US,en;q=0.9,ar-LB;q=0.8,ar;q=0.7",
        "cache-control": "max-age=0"}


    r = None
    proxies = {
        'http': 'socks5://127.0.0.1:9050',
        'https': 'socks5://127.0.0.1:9050'
    }
    try:
        r = s.get(url, headers=headers)

        if r.status_code == 403:
            i = 1
            while r.status_code == 403 and i <= 10:
                print("Status code: " + str(r.status_code) + "  for : " + url)
                time.sleep(1)
                r = s.get(url, headers=headers, proxies = proxies)
                i += 1
        # print("Status code: " + str(r.status_code) + "  for : " + url)
        # r.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print("Http Error: " + errh + " for url: " + url)
        if r.get_status_code == 403:
            i = 1
            while r.status_code == 403 and i <= 10:
                time.sleep(i)
                r = s.get(url, headers=headers)
                i += 1
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting: " + str(errc) + "  for : " + url)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error: " + str(errt) + "  for : " + url)
    except requests.exceptions.RequestException as err:
        print("OOps: Something Else" + str(err) + "  for : " + url)

    return r




def doGet13F(url, payload=None):
    s = requests.Session()
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'Sec-CH-UA': 'Examplary Browser',
        'Sec-CH-UA-Mobile': '?0',
        'Sec-CH-UA-Platform': "Windows",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "authority": "www.sec.gov",
        "method": "GET",
        "path": "/Archives/edgar/data/59478/000120919121046268/0001209191-21-046268-index.htm",
        "scheme": "https",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip deflate br",
        "accept-language": "en-US,en;q=0.9,ar-LB;q=0.8,ar;q=0.7",
        "cache-control": "max-age=0"}

    r = None
    proxies = {
        'http': 'socks5://127.0.0.1:9050',
        'https': 'socks5://127.0.0.1:9050'
    }

    try:
        if payload:
            r = s.get(url, headers=headers, params=payload)
        else:
            r = s.get(url, headers=headers)

        if r.status_code == 403:
            i = 1
            while r.status_code == 403 and i <= 10:
                print("Status code: " + str(r.status_code) + " for: " + url)
                time.sleep(1)
                if payload:
                    r = s.get(url, headers=headers, params=payload, proxies=proxies)
                else:
                    r = s.get(url, headers=headers, proxies=proxies)
                i += 1
    except requests.exceptions.HTTPError as errh:
        print("Http Error: " + str(errh) + " for url: " + url)
        if r.status_code == 403:
            i = 1
            while r.status_code == 403 and i <= 10:
                time.sleep(i)
                if payload:
                    r = s.get(url, headers=headers, params=payload)
                else:
                    r = s.get(url, headers=headers)
                i += 1
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting: " + str(errc) + " for: " + url)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error: " + str(errt) + " for: " + url)
    except requests.exceptions.RequestException as err:
        print("OOps: Something Else" + str(err) + " for: " + url)

    return r

# https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&CIK=&type=13F-HR&company=&dateb=&owner=include&start=0&count=40&output=atom


def download_filings(start_date, end_date):
    base_url = "https://www.sec.gov"
    search_url = f"{base_url}/cgi-bin/browse-edgar"

    # Format the dates in yyyy-mm-dd format
    start_date = start_date
    end_date = end_date
    start_page = 0
    # Create the search payload with the specified parameters (try both getcurrent and getgeneral)
    payload = {
        "action": "getcurrent",
        "CIK": "",
        "company":"",
        "SIC":"",
        "state":"",
        "type": "13F-HR",
        "datea": start_date,
        "dateb": end_date,
        "owner": "include",
        "accno":"",
        "output": "atom",
        "count": "100",
        "start":start_page
    }

    # Store all the filings URLs
    filings_urls = []

    # Store the undownloaded filings URLs list
    to_be_downloaded_filings = []


    while True:
        payload["start"] = str(start_page)
        response = doGet13F(search_url, payload=payload)
        response_str = response.text
        soup = BeautifulSoup(response_str, 'xml')

        # Find all the entries and extract the filing URLs
        entries = soup.find_all("entry")
        if not entries:
            break

        for entry in entries:
            link = entry.find("link", attrs={"type": "text/html"})
            title = entry.find("title")
            filing_type = title.text.strip()
            # print(f'Filing Type Before Verification: {filing_type}')
            if filing_type.startswith("13F-HR") and not filing_type.startswith("13F-HR/A") and not filing_type.startswith("13F-NT"):
                # print(f'Filing Type Being Selected: {filing_type}')       
                filings_urls.append(link.get("href"))
                # print(f'Filings URLs length: {len(filings_urls)}')


        start_page += 100


    print(f'Total filings from {start_date} to {end_date}: {len(filings_urls)}')

    # Create the directory to save the filings
    directory = "data/13F"
    os.makedirs(directory, exist_ok=True)
    # processed_directory = 'data/13F_processed'
    # # Create the processed directory if it doesn't exist
    # os.makedirs(processed_directory, exist_ok=True)

    # Screen for new filings
    for filing_url in filings_urls:
        filing_id = filing_url.split("/")[-1].split(".")[0]
        filing_id = filing_id.replace("-index", "")

        # Extract the year from the filing_id using regex
        pattern = r'-(\d{2})-'
        match = re.search(pattern, filing_id)
        
        if match:
            year_number = match.group(1)
            filing_id = filing_id.split('.')[0]

            # Convert the year number to a four-digit year
            if int(year_number) >= 95 and int(year_number) <= 99:
                year = 1900 + int(year_number)
            else:
                year = 2000 + int(year_number)

        filename = f"{directory}/{year}/{filing_id}.txt"  # Updated filename with subdirectory

        # Check if the filing has already been saved
        if os.path.exists(filename):
            # print(f"Filing {filing_id} already downloaded. Skipping.")
            continue
        else:
            to_be_downloaded_filings.append((filing_url,year))

    print(f'Total filings to be downloaded from {start_date} to {end_date}: {len(to_be_downloaded_filings)}')
        
    
    
    # Download new filings:
    for filing_data in to_be_downloaded_filings:
        filing_url, year = filing_data  # Unpack the tuple
        filing_id = filing_url.split("/")[-1].split(".")[0]
        filing_id = filing_id.replace("-index", "")
        filename = f"{directory}/{year}/{filing_id}.txt"
        print(f'filing url is: {filing_url}')
        response2 = doGet(filing_url)
        # print("Response2:", response2.status_code)  # Verify response status
        response2_str = response2.text
        soup = BeautifulSoup(response2_str, 'xml')

        # Find the document link and download the filing
        table = soup.find("table", class_="tableFile")
        # print(f'Table: {table}')
        rows = table.find_all("tr")
        # print(f'Rows: {rows}')
        for row in rows:
            cells = row.find_all("td")
            # print(f'Cells: {cells}')
            # if len(cells) > 3 and cells[3].text.strip() == "13F-HR":
            if len(cells) > 2 and "Complete submission text file" in cells[1].text:
                document_link = cells[2].find("a")
                if document_link:
                    document_url = base_url + document_link.get("href")
                    # print(f'Document URL: {document_url}')
                    document_response = doGet(document_url)

                    with open(filename, "wb") as file:
                        file.write(document_response.content)

                    # print(f"Downloaded filing: {filing_id}")
                    break  # Only download the first document link found

    print("All filings downloaded successfully.")


In [4]:
# to download in multiple day batches (change your timedelta accordingly). 
# Problem is during filing period, too many filings will be available that exceed the number of pages the feed will be able to get
def split_into_two_weeks(start_date, end_date):
    date_format = "%Y-%m-%d"
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)
    delta = timedelta(days=14)

    current_start = start
    current_end = min(start + delta, end + timedelta(days=1))  # Ensure the last interval doesn't exceed the end date

    while current_start <= end:
        download_filings(current_start.strftime(date_format), current_end.strftime(date_format))

        current_start += delta
        current_end = min(current_start + delta, end + timedelta(days=0))

# Example usage
start_date = "2023-07-10"
end_date = "2023-07-20"




split_into_two_weeks(start_date, end_date)






# Example usage
# download_filings("202-01-01", "2023-03-01")


Total filings from 2023-07-10 to 2023-07-21: 1286
Total filings to be downloaded from 2023-07-10 to 2023-07-21: 0
All filings downloaded successfully.


In [11]:
# To download day by day
def split_into_days(start_date, end_date):
    date_format = "%Y-%m-%d"
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)
    delta = timedelta(days=1)  # Change delta to 1 day

    current_date = start

    while current_date <= end:
        download_filings(current_date.strftime(date_format), current_date.strftime(date_format))

        current_date += delta

# Example usage
start_date = "2023-05-01"
end_date = "2023-06-01"

split_into_days(start_date, end_date)

Total filings from 2023-05-01 to 2023-05-01: 103
Total filings to be downloaded from 2023-05-01 to 2023-05-01: 0
All filings downloaded successfully.
Total filings from 2023-05-02 to 2023-05-02: 216
Total filings to be downloaded from 2023-05-02 to 2023-05-02: 0
All filings downloaded successfully.
Total filings from 2023-05-03 to 2023-05-03: 196
Total filings to be downloaded from 2023-05-03 to 2023-05-03: 0
All filings downloaded successfully.
Total filings from 2023-05-04 to 2023-05-04: 247
Total filings to be downloaded from 2023-05-04 to 2023-05-04: 0
All filings downloaded successfully.
Total filings from 2023-05-05 to 2023-05-05: 208
Total filings to be downloaded from 2023-05-05 to 2023-05-05: 0
All filings downloaded successfully.
Total filings from 2023-05-06 to 2023-05-06: 0
Total filings to be downloaded from 2023-05-06 to 2023-05-06: 0
All filings downloaded successfully.
Total filings from 2023-05-07 to 2023-05-07: 0
Total filings to be downloaded from 2023-05-07 to 2023-